# Face Re-Aging Demos with Gradio
To try out the Gradio Demos using Google Colab, run the cells below.
Be sure to select a runtime with GPU (e.g. `Runtime > Change Runtime Type > T4 (GPU)`) to make the model run fast.

You can choose to either run the regular inference demo or the video demo. The demo will open in a new tab.

## Downloading files and model

In [ ]:
!git clone https://github.com/timroelofs123/face_reaging

Cloning into 'face_reaging'...


In [ ]:
!git clone https://huggingface.co/timroelofs123/face_re-aging

In [ ]:
%mv face_re-aging/best_unet_model.pth face_reaging/
%cd face_reaging/

## Installing requirements

In [ ]:
%pip install gradio face_recognition antialiased_cnns kaleido av --quiet

## Running demo

Re-age your photo or video with the three demos.

In [ ]:
import gradio as gr
import torch
from model.models import UNet
from scripts.test_functions import process_image, process_video

In [ ]:
# default settings
window_size = 512
stride = 256
steps = 18
frame_count = 100
model_path = 'best_unet_model.pth'

In [ ]:
# The code from this block equivalent to the "run" function in scripts.gradio_demo

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(model_path, map_location=device))
unet_model.eval()

def block_vid(video_path, source_age, target_age):
    return process_video(unet_model, video_path, source_age, target_age,
                          window_size=window_size, stride=stride, frame_count=frame_count)

demo_vid = gr.Interface(
    fn=block_vid,
    inputs=[
        gr.Video(),
        gr.Slider(10, 90, value=20, step=1, label="Current age", info="Choose your current age"),
        gr.Slider(10, 90, value=80, step=1, label="Target age", info="Choose the age you want to become")
    ],
    outputs=gr.Video(),
    examples=[
        ['/content/video.mp4', 20, 60],
    ],
    description="Input a video of a person, and it will be aged frame-by-frame."
)

demo = gr.TabbedInterface([demo_vid],
                          tab_names=['Video inference demo'],
                          title="Face Re-Aging Demo",
                          )

In [ ]:
demo.launch()